In [1]:
import os 
os.chdir("../..")
os.getcwd()

'/home/h/uav-forests'

In [2]:
import numpy as np
import json
from detectron2.structures import BoxMode
import pandas as pd
import geopandas as gpd
import cv2
import matplotlib.pyplot as plt
import tifffile

from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from src.utils.dataset import split_train_val_test, get_detectron2_dataset_dicts#, #register_detectron2_datasets
import pickle

In [3]:
import os
import cv2
import pandas as pd
import geopandas as gpd
import numpy as np

from functools import partial

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

from sklearn.model_selection import train_test_split


def split_train_val_test(samples, train_ratio, val_ratio, test_ratio=None):
    test_ratio = test_ratio or (1.0 - train_ratio - val_ratio)
    assert 0 < train_ratio < 1
    assert 0 < val_ratio < 1
    assert 0 < test_ratio < 1
    assert 0 < train_ratio + val_ratio + test_ratio <=1

    train, valtest = train_test_split(samples, train_size=train_ratio)
    val, test = train_test_split(
        valtest, train_size=val_ratio / (val_ratio + test_ratio))

    return {"train": train,
            "val": val,
            "test": test}


def _convert_single_bbox(bbox):
    py, px = bbox.exterior.xy
    return {"bbox": [int(min(px)), int(min(py)),
                     int(max(px)), int(max(py))],
            "bbox_mode": BoxMode.XYXY_ABS,
            "category_id": 0}


def _convert_single_patch(patch_df, images_dir,image_shape=(256, 256)):
    patch_number = patch_df["patch_number"].iloc[0]
    file_path = os.path.join(images_dir, f"patch_{patch_number}.tif")
    image_shape = image_shape or cv2.imread(file_path).shape[:2]
    return {"file_name": file_path,
            "image_id": patch_number,
            "height": image_shape[0],
            "width": image_shape[1],
            "annotations": patch_df["bbox"].apply(_convert_single_bbox).tolist()}


# def get_detectron2_dataset_dicts(images_dir, patches_subset, min_bbox_area=100,
#                                  limit_train=-1, limit_val=-1, limit_test=-1):

   


In [4]:
images_dir = "../szprotawa_patches_256_rgb_ir/"
min_bbox_area = 400

annotations = pd.read_pickle(f"{images_dir}/annotation.pkl").set_geometry("bbox")

splits = split_train_val_test(annotations["patch_number"].unique(), 0.7, 0.15, 0.15)
patches_subset = splits['test']

In [5]:
annotations = annotations[annotations["bbox"].area >= min_bbox_area]
annotations = annotations[annotations["patch_number"].isin(patches_subset)]

In [6]:
g = annotations.groupby("patch_number")

In [7]:
dataset_dicts = get_detectron2_dataset_dicts(images_dir, splits["val"], min_bbox_area=200)#g.apply(_convert_single_patch, images_dir=images_dir, image_shape=(256,256)).tolist()

In [8]:
DatasetCatalog.register("szp_test3", lambda: dataset_dicts)

In [9]:
MetadataCatalog.get("szp_test3").set(thing_classes=["SickTrees"])

Metadata(name='szp_test3', thing_classes=['SickTrees'])

In [10]:
# dataset_dicts = get_szprotawa_dicts("data/szprotawa_patches_256/")
# dataset_dicts = DatasetCatalog.get("szprotawa_train")

In [11]:
meta = MetadataCatalog.get("szp_test3")

In [12]:
'''
smpl = np.random.choice(dataset_dicts, 3)
rgb_axes = [0,3]
ndvi_axes = [3,4]
for d, axes in zip(smpl, (rgb_axes , rgb_axes, rgb_axes)):
    print(d["file_name"])
    img = tifffile.imread(d["file_name"])
    
    img = np.swapaxes(img, 0, -1)
    
    if axes == ndvi_axes:
        img = np.reshape(img, img.shape[:2])
        img[:,axes[0]:axes[1]]
    else:
        img[:,:,axes[0]:axes[1]]
    #print(img.shape, img.dtype)
    
    visualizer = Visualizer(img, metadata=meta, 
                            scale=0.75)
    #vis = visualizer.draw_dataset_dict(d)
    plt.figure()
    plt.imshow(vis.get_image())
'''

'\nsmpl = np.random.choice(dataset_dicts, 3)\nrgb_axes = [0,3]\nndvi_axes = [3,4]\nfor d, axes in zip(smpl, (rgb_axes , rgb_axes, rgb_axes)):\n    print(d["file_name"])\n    img = tifffile.imread(d["file_name"])\n    \n    img = np.swapaxes(img, 0, -1)\n    \n    if axes == ndvi_axes:\n        img = np.reshape(img, img.shape[:2])\n        img[:,axes[0]:axes[1]]\n    else:\n        img[:,:,axes[0]:axes[1]]\n    #print(img.shape, img.dtype)\n    \n    visualizer = Visualizer(img, metadata=meta, \n                            scale=0.75)\n    #vis = visualizer.draw_dataset_dict(d)\n    plt.figure()\n    plt.imshow(vis.get_image())\n'

In [13]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

Reusing TensorBoard on port 6007 (pid 20186), started 1:19:47 ago. (Use '!kill 20186' to kill it.)

In [14]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

In [15]:
from datetime import datetime


In [16]:
cfg = get_cfg()

cfg.OUTPUT_DIR = f"./tboard_logs/retinanet {datetime.now().isoformat().rsplit(':', 1)[0]}/"
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("szp_test3",)
cfg.DATASETS.TEST = ("szp_test3",)
cfg.DATALOADER.NUM_WORKERS = 4

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10_000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset

cfg.MODEL.RETINANET.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.RETINANET.NUM_CLASSES = 1  # only has one class (ballon)

cfg.TEST.EVAL_PERIOD = 0#2_000 #200
cfg.TEST.DETECTIONS_PER_IMAGE = 5

cfg.VIS_PERIOD = 100
cfg.MODEL.PIXEL_MEAN = [103.53, 116.28, 123.675, 100.1] #ostatnia wartość wpisana z palca
cfg.MODEL.PIXEL_STD =  [1.0, 1.0, 1.0, 1.0] #ostatnia wartość wpisana z palca
print(cfg)

Loading config /home/h/detectron2/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('szp_test3',)
  TRAIN: ('szp_test3',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32, 40.3

In [17]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader


class Trainer(DefaultTrainer):
    
    
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return COCOEvaluator(dataset_name, cfg, False, output_dir=cfg.OUTPUT_DIR+"/eval/")



In [18]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

loss_box_reg: 0.007  time: 0.3710  data_time: 0.0068  lr: 0.000250  max_mem: 2539M
[02/09 18:16:50 d2.utils.events]: eta: 0:12:24  iter: 7999  total_loss: 0.012  loss_cls: 0.006  loss_box_reg: 0.007  time: 0.3710  data_time: 0.0066  lr: 0.000250  max_mem: 2539M
[02/09 18:16:57 d2.utils.events]: eta: 0:12:17  iter: 8019  total_loss: 0.013  loss_cls: 0.005  loss_box_reg: 0.007  time: 0.3710  data_time: 0.0070  lr: 0.000250  max_mem: 2539M
[02/09 18:17:05 d2.utils.events]: eta: 0:12:09  iter: 8039  total_loss: 0.013  loss_cls: 0.005  loss_box_reg: 0.007  time: 0.3710  data_time: 0.0070  lr: 0.000250  max_mem: 2539M
[02/09 18:17:12 d2.utils.events]: eta: 0:12:02  iter: 8059  total_loss: 0.012  loss_cls: 0.004  loss_box_reg: 0.007  time: 0.3710  data_time: 0.0070  lr: 0.000250  max_mem: 2539M
[02/09 18:17:19 d2.utils.events]: eta: 0:11:54  iter: 8079  total_loss: 0.012  loss_cls: 0.005  loss_box_reg: 0.007  time: 0.3710  data_time: 0.0068  lr: 0.000250  max_mem: 2539M
[02/09 18:17:27 d2.uti

In [19]:
cfg.OUTPUT_DIR

'./tboard_logs/retinanet 2020-02-09T17:27/'

In [20]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.3
cfg.DATASETS.TEST = ("szp_test3", )
cfg.TEST.SCORE_THRESHOLD = 0.3
cfg.TEST.SCORE_THRESH = 0.3

cfg.TEST.DETECTIONS_PER_IMAGE = 10
predictor = DefaultPredictor(cfg)

predictor.cfg["TEST"]

CfgNode({'EXPECTED_RESULTS': [], 'EVAL_PERIOD': 0, 'KEYPOINT_OKS_SIGMAS': [], 'DETECTIONS_PER_IMAGE': 10, 'AUG': CfgNode({'ENABLED': False, 'MIN_SIZES': (400, 500, 600, 700, 800, 900, 1000, 1100, 1200), 'MAX_SIZE': 4000, 'FLIP': True}), 'PRECISE_BN': CfgNode({'ENABLED': False, 'NUM_ITER': 200}), 'SCORE_THRESHOLD': 0.3, 'SCORE_THRESH': 0.3})

In [21]:
1+2

3

In [22]:
from detectron2.utils.visualizer import ColorMode
# dataset_dicts = DatasetCatalog.get("szprotawa_val")

In [23]:

for d in np.random.choice(dataset_dicts, 1):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1].copy(),
                   metadata=meta, 
                   scale=0.8)
    vis = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    f, a = plt.subplots(1,2, figsize=(10,5))
    a[0].imshow(vis.get_image())
    
    v = Visualizer(im[:,:,::-1].copy(),
                   metadata=meta, 
                   scale=0.8)
    vis = v.draw_dataset_dict(d)
    a[1].imshow(vis.get_image())

RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 0

In [0]:
_ = register_detectron2_datasets("szprott", "./data/szprotawa_patches_256/", 0.7, 0.15, 0.15, 100, {'test': splits['test'][:500]})

In [0]:
from detectron2.evaluation import DatasetEvaluator, COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader



evaluator = COCOEvaluator("szprottx_test", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "szprott_test")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

In [0]:
1+2

In [0]:
outputs.keys()